In [ ]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function, division, absolute_import
import collections
import math
import numpy as np
import os
import random
import tensorflow as tf
import zipfile
from matplotlib import pyplot as plt
from matplotlib import pylab
from six.moves import range
from six.moves.urllib.request import urlretrieve
from sklearn.manifold import TSNE
import time
import pandas as pd
import warnings
import scipy.interpolate as interpolate
from matplotlib.colors import Normalize
import matplotlib.cm as cm
%matplotlib inline

In [ ]:
# custom imports
import spike_heatmap

In [ ]:
base_path = "./"
template_marking = np.load(os.path.join(base_path, 'template_marking.npy'))
spike_times = np.load(os.path.join(base_path, 'spike_times.npy'))
spike_templates = np.load(os.path.join(base_path, 'spike_templates.npy'))
types = {0: 'Noise', 1: 'SS', 2: 'SS_Contaminated', 3: 'SS_Putative', 4: 'MUA', 5: 'Unspesified_1',
             6: 'Unspecified_2',
             7: 'Unspecified_3'}

In [ ]:
avg_spike_template = np.load(os.path.join(base_path, 'avg_spike_template.npy'))
# avg_spike_template

In [ ]:
avg_spike_template.shape


plt.plot(avg_spike_template[55,...].T)

In [ ]:
prb_path = "prb.txt"

connected = np.squeeze(np.load(os.path.join(base_path, 'channel_map.npy')))
number_of_channels_in_binary_file = 1440
connected_binary = np.in1d(np.arange(number_of_channels_in_binary_file), connected)
bad_channels = np.squeeze(np.argwhere(connected_binary == False).astype(np.int))

In [ ]:
def get_loc(template, plot=False):
  _, _, _, _, p2p = spike_heatmap.peaktopeak(template, window_size=60)
  zlimits = [p2p.min(), p2p.max()]
  probe = spike_heatmap.get_probe_geometry_from_prb_file(prb_path)
  num_of_shanks = 1
  channel_positions = pd.Series(probe[0]['geometry'])
  channel_positions = channel_positions.drop(bad_channels)
  channel_positions.index = np.arange(len(channel_positions))
  total_electrodes = len(channel_positions)
  electrodes_per_shank = int(total_electrodes / num_of_shanks)
  begin_electrode = 0
  end_electrode = electrodes_per_shank
  channel_positions_shank = channel_positions[begin_electrode:end_electrode]
  data = p2p[channel_positions_shank.index]

  hpos = 0
  vpos = 0
  width = None
  height = None
  gridscale = 1
  interpolation_method = "bicubic"

  if np.isnan(data).any():
    warnings.warn('The data passed to contain NaN values. \
    These will create unexpected results in the interpolation. \
    Deal with them.')
  channel_positions = channel_positions.sort_index(ascending=[1])
  channel_positions = np.array([[x, y] for x, y in channel_positions.values])

  if not width and not height:
    x_scaling = 1
    y_scaling = 1

  chan_x = channel_positions[:, 0] * x_scaling + hpos
  chan_y = channel_positions[:, 1] * y_scaling + vpos
  chan_x = np.max(chan_x) - chan_x

  hlim = [np.min(chan_y), np.max(chan_y)]
  vlim = [np.min(chan_x), np.max(chan_x)]

  yi, xi = np.mgrid[hlim[0]:hlim[1]:complex(0, gridscale)*(hlim[1]-hlim[0]),
                    vlim[0]:vlim[1]:complex(0, gridscale)*(vlim[1]-vlim[0])]

  zi = interpolate.griddata((chan_x, chan_y), data, (xi, yi))
  

  if zlimits is None:
      vmin = data.min()
      vmax = data.max()
  else:
      vmin = zlimits[0]
      vmax = zlimits[1]
  if plot:
    cmap = plt.get_cmap("jet")
    plt.figure(figsize=(20,70))
    image = plt.imshow(zi.T, cmap=cmap, origin=['lower'], vmin=vmin, 
                        vmax=vmax, interpolation=interpolation_method,
                        extent=[hlim[0], hlim[1], vlim[0], vlim[1]],
                        aspect='equal').make_image(renderer=None)
    plt.show() 
  return plt.gcf(), zi[:-1,:]

In [ ]:
print("spike_template shape is {0}".format(spike_templates.shape))
print("spike_template max is {0}".format(spike_templates.max()))
print("template_marking shape is {0}".format(template_marking.shape))
print("spike_template itself is:")
template_marking.shape

In [ ]:
start_t = time.time()
spike_times_used = list()
spike_templates_used = list()
for idx, cell in enumerate(spike_templates):
  if template_marking[cell]==1:
    spike_times_used.append(spike_times[idx])
    spike_templates_used.append(cell)
print(time.time()-start_t)

In [ ]:
spike_times_used = np.array(spike_times_used)
spike_templates_used = np.array(spike_templates_used)
print(spike_times_used.shape)

In [ ]:
np.diff(spike_times_used.squeeze()).mean()

In [ ]:
unique = np.unique(spike_templates_used)
unique.shape

In [ ]:
cell_pos = dict()
location_array = np.arange(3589)
for i, template in enumerate(unique[:10]):
  mplot, heat = get_loc(avg_spike_template[template,...], plot=True)
  tmp = np.sum(heat, axis=1)
  tmp_max = tmp.max()
  tmp[tmp<0.8*tmp_max]=0
  cell_pos[template] = np.dot(tmp/tmp.sum(),
                              location_array)
  tmp[3300:]=0
  if (tmp.max()> tmp.mean()+3*tmp.std()) and (tmp.sum() > 0):
    cell_pos[template] = np.dot(tmp/tmp.sum(),
                                location_array)
  print(cell_pos[template])

In [ ]:
cell_pos = dict()

for i, template in enumerate(unique):
  mplot, heat = get_loc(avg_spike_template[template,...])
  tmp = np.sum(heat, axis=1)
  tmp_max = tmp.max()
  tmp[tmp<0.8*tmp_max]=0
  cell_pos[template] = np.dot(tmp/tmp.sum(),
                              location_array)
  tmp[3300:]=0
  if (tmp.max()> tmp.mean()+3*tmp.std()) and (tmp.sum() > 0):
    cell_pos[template] = np.dot(tmp/tmp.sum(),
                                location_array)
  if i%100==0:
    print(i)

neuron2vec
=============

with little modification from udacity tutorial
------------

The goal of this notebook is to train a Neuron2Vec skip-gram model over spiking data of chronic neuroseeker experiment.

In [ ]:
spike_times_used.shape

Build the dictionary and replace rare templates with 5000 token.

In [ ]:
vocabulary_size = 1200

def build_dataset(words):
  count = [[5000, -1]]
#   count = list()
  count.extend(collections.Counter(words).most_common(vocabulary_size - 1))
  dictionary = dict()
  for word, _ in count:
    dictionary[word] = len(dictionary)
  data = list()
  unk_count = 0
  for word in words:
    if word in dictionary:     #r
      index = dictionary[word]
    else:
      index = 0  # dictionary['UNK']#r
      unk_count = unk_count + 1#r
    data.append(index)
  count[0][1] = unk_count#r
  reverse_dictionary = dict(zip(dictionary.values(), dictionary.keys()))
  return data, count, dictionary, reverse_dictionary

data, count, dictionary, reverse_dictionary = build_dataset(spike_templates_used.astype(np.uint16).squeeze().tolist())
# data, count, dictionary, reverse_dictionary = build_dataset(words)
print('Most common words + UNK', count[:5])
print('Sample data', data[:10])
# del words  # Hint to reduce memory.

Function to generate a training batch for the skip-gram model.

In [ ]:
data_index = 0

def generate_batch(batch_size, num_skips, skip_window):
  global data_index
  assert batch_size % num_skips == 0
  assert num_skips <= 2 * skip_window
  batch = np.ndarray(shape=(batch_size), dtype=np.int32)
  labels = np.ndarray(shape=(batch_size, 1), dtype=np.int32)
  span = 2 * skip_window + 1 # [ skip_window target skip_window ]
  buffer = collections.deque(maxlen=span)
  for _ in range(span):
    buffer.append(data[data_index])
    data_index = (data_index + 1) % len(data)
  for i in range(batch_size // num_skips):
    target = skip_window  # target label at the center of the buffer
    targets_to_avoid = [ skip_window ]
    for j in range(num_skips):
      while target in targets_to_avoid:
        target = random.randint(0, span - 1)
      targets_to_avoid.append(target)
      batch[i * num_skips + j] = buffer[skip_window]
      labels[i * num_skips + j, 0] = buffer[target]
    buffer.append(data[data_index])
    data_index = (data_index + 1) % len(data)
  return batch, labels

print('data:', [reverse_dictionary[di] for di in data[:8]])

for num_skips, skip_window in [(2, 1), (4, 2)]:
    data_index = 0
    batch, labels = generate_batch(batch_size=8, num_skips=num_skips, skip_window=skip_window)
    print('\nwith num_skips = %d and skip_window = %d:' % (num_skips, skip_window))
    print('    batch:', [reverse_dictionary[bi] for bi in batch])
    print('    labels:', [reverse_dictionary[li] for li in labels.reshape(8)])

In [ ]:
batch.shape

Train a skip-gram model.

In [ ]:
batch_size = 256
embedding_size = 20 # Dimension of the embedding vector.
skip_window = 50 #3 # How many words to consider left and right.
num_skips = 64 # How many times to reuse an input to generate a label.
num_sampled = 128 # Number of negative examples to sample.

graph = tf.Graph()


with graph.as_default(), tf.device('/cpu:0'):

  # Input data.
  train_dataset = tf.placeholder(tf.int32, shape=[batch_size])
  train_labels = tf.placeholder(tf.int32, shape=[batch_size, 1])
  
  # Variables.
  embeddings = tf.Variable(
    tf.random_uniform([vocabulary_size, embedding_size], -1.0, 1.0))
  softmax_weights = tf.Variable(
    tf.truncated_normal([vocabulary_size, embedding_size],
                         stddev=1.0 / math.sqrt(embedding_size)))
  softmax_biases = tf.Variable(tf.zeros([vocabulary_size]))
  
  # Model.
  # Look up embeddings for inputs.
  embed = tf.nn.embedding_lookup(embeddings, train_dataset)
    
  nce_weights = tf.Variable(
      tf.truncated_normal([vocabulary_size, embedding_size],
                          stddev=1.0 / math.sqrt(embedding_size)))
  nce_biases = tf.Variable(tf.zeros([vocabulary_size]))

# Compute the average NCE loss for the batch.
# tf.nce_loss automatically draws a new sample of the negative labels each
# time we evaluate the loss.
# Explanation of the meaning of NCE loss:
#   http://mccormickml.com/2016/04/19/word2vec-tutorial-the-skip-gram-model/
  loss = tf.reduce_mean(
      tf.nn.nce_loss(weights=nce_weights,
                     biases=nce_biases,
                     labels=train_labels,
                     inputs=embed,
                     num_sampled=num_sampled,
                     num_classes=vocabulary_size))

  # Optimizer.
  # Note: The optimizer will optimize the softmax_weights AND the embeddings.
  # This is because the embeddings are defined as a variable quantity and the
  # optimizer's `minimize` method will by default modify all variable quantities 
  # that contribute to the tensor it is passed.
  # See docs on `tf.train.Optimizer.minimize()` for more details.
#   optimizer = tf.train.AdagradOptimizer(1.0).minimize(loss)
  optimizer = tf.train.AdamOptimizer(1e-3).minimize(loss)
  
  # Compute the similarity between minibatch examples and all embeddings.
  # We use the cosine distance:
  norm = tf.sqrt(tf.reduce_sum(tf.square(embeddings), 1, keep_dims=True))
  normalized_embeddings = embeddings / norm

In [ ]:
num_steps = 60001
prng=np.random.RandomState(0)
idxs = np.arange(batch_size)

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print('Initialized')
  average_loss = 0
  for step in range(num_steps):
    prng.shuffle(idxs)
    batch_data, batch_labels = generate_batch(
      batch_size, num_skips, skip_window)
    batch_data = batch_data[idxs]
    batch_labels = batch_labels[idxs]
    feed_dict = {train_dataset : batch_data, train_labels : batch_labels}
    _, l = session.run([optimizer, loss], feed_dict=feed_dict)
    average_loss += l
    if step % 2000 == 0:
      if step > 0:
        average_loss = average_loss / 2000
      # The average loss is an estimate of the loss over the last 2000 batches.
      print('Average loss at step %d: %f' % (step, average_loss))
      average_loss = 0
  final_embeddings = normalized_embeddings.eval()

In [ ]:
num_points = 200

tsne = TSNE(perplexity=30, n_components=2, 
            init='pca', n_iter=6000, method='exact',
            verbose=2)
two_d_embeddings = tsne.fit_transform(final_embeddings[1:num_points+1, :])

In [ ]:

cmap = cm.rainbow
norm = Normalize(vmin=min(cell_pos.values()),
                 vmax=max(cell_pos.values()))


In [ ]:
count_dic = {k:v for k,v in count}

In [ ]:
count_dic[5000]

In [ ]:

def plot(embeddings, labels):
  assert embeddings.shape[0] >= len(labels), 'More labels than embeddings'
  plt.figure(figsize=(15,15))  # in inches
  mcolors = list()
  mcount = list()
  for i, label in enumerate(labels):
    
    mcolors.append(norm(cell_pos[label]))
    mcount.append(str(count_dic[label]))
    x, y = embeddings[i,:]
#     plt.annotate(label, xy=(x, y), xytext=(5, 2), textcoords='offset points',
#                    ha='right', va='bottom')
    
  ticks = [1050/8100, 1930/8100, 3500/8100, 4450/8100, 5760/8100, 6230/8100, 8000/8100]  
  mycolors = np.digitize(np.array(mcolors), ticks)
    
  plt.scatter(embeddings[:,0], embeddings[:,1], c=[ticks[c] for c in mycolors], cmap=cmap)
#   plt.scatter(embeddings[:,0], embeddings[:,1], c=mcolors, cmap=cmap) continuous colors
  cb = plt.colorbar(ticks=[1050/8100, 1930/8100, 3500/8100, 4450/8100, 5760/8100, 6230/8100, 8000/8100])
  cb.ax.set_yticklabels(['SubThalamic', ' Thalamus VPM', 'Thalamus Posterior', 
                         'Thalamus LPMR', 'Hypocampus DG', 'Hypocampus CA1', 'Parietal Cortex'])
  return plt.gcf()


words = [reverse_dictionary[i] for i in range(1, num_points+1)]
tsne_fig = plot(two_d_embeddings, words)

In [ ]:
tsne_fig.savefig('tsne_20.jpg', format='jpg')